### Installation des dépendances

In [ ]:
%pip install numpy
%pip install opencv-python
%pip install -U matplotlib
import numpy as np
import cv2 as cv
from matplotlib import pyplot as plt
import math
import os
import time

In [2]:
# Define Constants
STEP_TO_NEXT_FRAME = 16 
""" in each video, we will read 1 frame every 24/STEP_TO_INDEX frames"""
DEBUG = False # if True, print debug information
NUMBER_OF_VIDEO = 100 # number of videos to index
VIDEOS_SIZE_ON_DISK = 444039168  # in octets
MAX_FRAME_NUMBER = 37895 # maximum number of frames in a video
N = int(MAX_FRAME_NUMBER/STEP_TO_NEXT_FRAME) # index Length
D = 8 # index Depth

In [3]:
#Define all global variables
imageList = []
videoList = []
indexationTable = [] # Table of indexation
Descriptor_list = np.empty((N, D * 3), dtype=np.float32) # Descriptor of the image

### Lecture des fichiers

In [4]:
imagePath = "/data/jpeg/"
videoPath = "/data/mp4/"
currDirectory = os.getcwd()
imageList = os.listdir(currDirectory + imagePath)
videoList = os.listdir(currDirectory + videoPath)

In [5]:
def getVideoParameter(videoPath:str):
    """return the frame rate and the number of frames of the video."""
    cap = cv.VideoCapture(videoPath)
    fps = cap.get(cv.CAP_PROP_FPS)
    frame_count = cap.get(cv.CAP_PROP_FRAME_COUNT)
    return fps, frame_count
    

In [6]:
def displayRGBHistogram(histogram_r, histogram_g, histogram_b):
    plt.plot(histogram_b, color='blue', label='Canal bleu')
    plt.plot(histogram_g, color='green', label='Canal vert')
    plt.plot(histogram_r, color='red', label='Canal rouge')
    plt.xlabel('Intensité des pixels')
    plt.ylabel('Nombre de pixels')
    plt.title('Histogramme de l\'image')
    plt.legend()
    plt.show()

In [7]:

def convertImageToVector(imagePath, needToRead:bool = True):
    """return the histogram of the image."""
    if needToRead:
        image = cv.imread(imagePath)
    else:
        image = imagePath
    
    #Separate the color channels
    canal_b = image[:, :, 0]
    canal_g = image[:, :, 1]
    canal_r = image[:, :, 2]

    # Calculate histograms for each channel
    # D means we will have 256/D values in the histogram
    histogram_b = cv.calcHist([canal_b], [0], None, [D], [0, 256])
    histogram_g = cv.calcHist([canal_g], [0], None, [D], [0, 256])
    histogram_r = cv.calcHist([canal_r], [0], None, [D], [0, 256])
    
    vector_r = cv.normalize(histogram_r, histogram_r).flatten()
    vector_g = cv.normalize(histogram_g, histogram_g).flatten()
    vector_b = cv.normalize(histogram_b, histogram_b).flatten()
    
    histogramme_complet = np.concatenate((vector_r, vector_g, vector_b))
    histogramme_complet = cv.normalize(histogramme_complet, histogramme_complet)
    
    if DEBUG:
        displayRGBHistogram(histogram_r, histogram_g, histogram_b)

    return histogramme_complet

In [8]:
def createIndexTable():
    """Index a video by creating a list of frames and their corresponding index. and update the indexationTable variable."""
    for video in videoList:
        path = currDirectory + videoPath + video
        fps, frame_count= getVideoParameter(path)
        cap = cv.VideoCapture(path)
        startIndex = len(indexationTable)
        stopIndex = startIndex - 1 + math.floor(frame_count/STEP_TO_NEXT_FRAME)
        for i in range(startIndex, stopIndex):
            frameNumber = STEP_TO_NEXT_FRAME*(i-startIndex)
            indexationTable.append((i, video, (frameNumber/fps)))
            currImage = cap.set(cv.CAP_PROP_POS_FRAMES, frameNumber)
            ret, currImage = cap.read()
            Descriptor_list[i] = convertImageToVector(currImage, False)
        
createIndexTable()

In [9]:
def CalculateCompressionRate():
    print(len(indexationTable))
    print(Descriptor_list.shape)
    rate = 1- (3*D*len(indexationTable)*4)/VIDEOS_SIZE_ON_DISK #*4 because we use float32 and 3 because we have 3 channels RGB
    print(rate)
    
CalculateCompressionRate()

2222
(2368, 24)
0.9994880451627186


In [10]:
def plotImage(image, type):
    figure = plt.figure(figsize = (10,10))
    imageout = np.clip(image,0,255)
    imageout= imageout.astype(type)
    plt.imshow(imageout, cmap='gray')
    plt.show()

In [11]:
def searchImage():
    SearchResult = []
    for image in imageList:
        path = currDirectory + imagePath + image
        if DEBUG:
            plotImage(cv.cvtColor(cv.imread(path), cv.COLOR_BGR2RGB), 'uint8')
        imageVector = convertImageToVector(path)
        for i in range(len(indexationTable)):
            # calculer la norme de la différence entre l'image et la liste de descripteurs
            dist = cv.compareHist(imageVector, Descriptor_list[i], cv.HISTCMP_BHATTACHARYYA)
            # dist = np.linalg.norm(imageVector-Descriptor_list[i])
            if i == 0:
                bestDistance = dist
                bestIndex = i
            if dist < bestDistance:
                bestDistance = dist
                bestIndex = i

        if bestDistance < 0.116:
            bestVideo = indexationTable[bestIndex][1]
            bestTime = indexationTable[bestIndex][2]

        else:
            bestVideo = "out"
            bestTime = ""
            
        # print("The best match is the frame number ", (image.split(".")[0], bestVideo.split(".")[0], bestTime)) 
        if type(bestTime) == float: 
            SearchResult.append((image.split(".")[0], bestVideo.split(".")[0], round(bestTime, 3)))
        else:
            SearchResult.append((image.split(".")[0], bestVideo.split(".")[0], bestTime)) 
        # break
    return SearchResult

SearchResult = searchImage()
    


In [12]:
def writeResultInCSVFile():
    """Function to write the results in a csv file."""
    with open(currDirectory+'/src/test.csv', 'w') as file:
        file.write("image,video_pred,minutage_pred\n")
        for imageName, video, time in SearchResult:
            file.write(imageName + "," + video + "," + str(time) + "\n")
    file.close()

writeResultInCSVFile()

In [13]:
# Affichage des résultats
import subprocess
directory = r'C:\Users\loicn\OneDrive - polymtl.ca\Polymtl\Session_5\INF8770_TEC_MULTIMEDIA\Lab\Lab3\src'
output = subprocess.run(['python3', 'evaluate.py', '--file', 'test.csv', '--file_gt', '../data/gt.csv'], cwd=directory, capture_output=True, text=True)

print(output.stdout)

Taux de bonnes rÃ©ponses : 92.6% (926/1000)
Ecart temporel moyen : 1.47 sec

